In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re, json

In [ ]:
#Carrega como objeto request para mudar o header.
req = Request('https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250', None, {'User-agent' : 'Mozilla/5.0 (Windows; U; Windows NT 5.1; de; rv:1.9.1.5) Gecko/20091102 Firefox/3.5.5'})
html = urlopen(req)

bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)
for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

In [ ]:
print(bs)

In [ ]:
tudo = bs.find_all('a', href=True)
links0 = [link['href'] for link in tudo if re.match(r'^/title/tt\d+/\?ref_=chttvtp_i_\d+$', link['href'])]

links = ['https://www.imdb.com/' + link for link in links0]
print(links)


In [ ]:
titulos = []
anos = []
episodios = []
notas = []

tag = bs.find('script', {'id': '__NEXT_DATA__'})

if tag:

    data0 = tag.contents[0]

    data = json.loads(data0)

    for edge in data['props']['pageProps']['pageData']['chartTitles']['edges']:
        titulo = edge['node']['titleText']['text']
        ano = edge['node']['releaseYear']['year']
        episodio = edge['node']['episodes']['episodes']['total']
        nota = edge['node']['ratingsSummary']['aggregateRating']
        titulos.append(titulo)
        anos.append(ano)
        episodios.append(episodio)
        notas.append(nota)
else:
    print("Erro ao encontrar o script JSON")


In [ ]:
dados = {
    "titulo": titulos,
    "anos": anos,
    "episodios": episodios,
    "notas": notas,
    "links": links
}
json_dados = json.dumps(dados, indent=4)

print(json_dados)

In [7]:
class Serie:
    def __init__(self, popularidade, generos, elenco_principal):
        self.popularidade = popularidade
        self.generos = generos
        self.elenco_principal = elenco_principal



In [ ]:
series = []
for link in links:
  url = link
  req = Request(url, None, {'User-agent' : 'Mozilla/5.0 (Windows; U; Windows NT 5.1; de; rv:1.9.1.5) Gecko/20091102 Firefox/3.5.5'})
  html = urlopen(req)
  bs = BeautifulSoup(html.read(), 'html.parser')

  popularidade0 = bs.find('div', {'data-testid':'hero-rating-bar__popularity__score'})
  popularidade = popularidade0.text.strip()

  generos_div = bs.find('div', class_='ipc-chip-list__scroller')
  generos_links = generos_div.find_all('a')
  generos = [link.find('span', class_='ipc-chip__text').text.strip() for link in generos_links]

  elenco_0 = bs.find_all('div', {'data-testid': 'title-cast-item'})
  elenco = []
  for i in elenco_0:
    ator_link = i.find('a', {'data-testid': 'title-cast-item__actor'})
    elenco.append(ator_link.text.strip())
  series.append(Serie(popularidade, generos, elenco))
  break#Demora muito
json_dados = json.dumps([serie.__dict__ for serie in series], indent=4)
print(json_dados)
